In [1]:
import pandas as pd
import numpy as np
import re
import math
import tabula
import camelot
import pdfminer
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.converter import TextConverter
from pdfminer.layout import LAParams
from pdfminer.pdfpage import PDFPage
try:
    from cStringIO import StringIO
except ImportError:
    from io import StringIO

In [101]:
1,2
f = r'C:\Users\MudraCircle\Desktop\bks_raw\Parsing_testing\KOTAK\files\type1\kotak02.pdf'

In [102]:
tables = camelot.read_pdf(f, flavor='stream', pages='all', edge_tol=500)
tables

<TableList n=5>

In [160]:
if len(tables) != 0:
    dt = pd.DataFrame(); tmp = pd.DataFrame();
    for i in range(len(tables)):
        tmp = tables[i].df
        if (tmp.shape[1]<5):
            print(i, ": Less than five")
            pass
        elif (tmp.shape[1]==9):
            try:
                idx=[ c for c in tmp[tmp.apply(lambda row: row.astype(str).str.lower().str.contains('balance').any(), axis=1) ==True].index if c in tmp[tmp.apply(lambda row: row.astype(str).str.lower().str.contains('date').any(), axis=1) ==True].index ][0]
                tmp.columns=tmp.iloc[idx] ; tmp=tmp.iloc[idx+1:,:] ; tmp.reset_index(drop=True,inplace=True)           
            except:
                try:
                    idx=[ c for c in tmp[tmp.apply(lambda row: row.astype(str).str.lower().str.contains('balance').any(), axis=1) ==True].index if c in tmp[tmp.apply(lambda row: row.astype(str).str.lower().str.contains('description').any(), axis=1) ==True].index ][0]
                    tmp.columns=tmp.iloc[idx] ; tmp=tmp.iloc[idx+1:,:] ; tmp.reset_index(drop=True,inplace=True)           
                except:
                    print("\nHtmpC-Column headers missing"); pass
            dt=pd.concat([dt,tmp]).drop_duplicates(keep='first').reset_index(drop=True)
        else:
            print("\nDifferent structure found"); pass;

print(len(dt.columns))
        
try:        
    if dt.columns[2] == "Date":
        dt.drop('Date', axis=1, inplace=True)
        dt.rename(columns={dt.columns[1] : "Date"}, inplace=True)
except Exception as e:
    print("Error : ",e)
        
dt.to_csv("check.csv", index=0)

4 : Less than five
9


In [133]:
df2 = dt
df2.head()

,Sl. No.,Date,Description,Chq / Ref number,Amount,Dr / Cr,Balance,Dr / Cr
0,,,IMPS from VARUN GHEI Ref,IMPS-912416820045,"10,000.00",CR,"69,300.17",CR
1,1,04/05/2019,,,,,,
2,,,912416332500,,,,,
3,,,MB:IMPS to 017101516788,IMPS-912416823590,"10,000.00",DR,"59,300.17",CR
4,2,04/05/2019,,,,,,


In [134]:
dt = df2
dt.head()

,Sl. No.,Date,Description,Chq / Ref number,Amount,Dr / Cr,Balance,Dr / Cr
0,,,IMPS from VARUN GHEI Ref,IMPS-912416820045,"10,000.00",CR,"69,300.17",CR
1,1,04/05/2019,,,,,,
2,,,912416332500,,,,,
3,,,MB:IMPS to 017101516788,IMPS-912416823590,"10,000.00",DR,"59,300.17",CR
4,2,04/05/2019,,,,,,


In [135]:
dt = dt[~dt.index.isin(dt[dt.apply(lambda row: row.astype(str).str.lower().str.contains('page:|account status|total|reason for return|inward clg|opening balance|statement of a/c|statement summary|generated on|generated by|computer generated statement|not').any(), axis=1) ==True].index)]
dt.drop(dt.nunique(dropna=False)[(dt.nunique(dropna=False) == 1)].index, axis=1,inplace=True)             
dt.iloc[:,-1]=dt.iloc[:,-1].fillna(dt.iloc[:,-2])

In [136]:
dt.head()

,Sl. No.,Date,Description,Chq / Ref number,Amount,Dr / Cr,Balance,Dr / Cr
0,,,IMPS from VARUN GHEI Ref,IMPS-912416820045,"10,000.00",CR,"69,300.17",CR
1,1,04/05/2019,,,,,,
2,,,912416332500,,,,,
3,,,MB:IMPS to 017101516788,IMPS-912416823590,"10,000.00",DR,"59,300.17",CR
4,2,04/05/2019,,,,,,


In [137]:
def abc(a):
    if type(a) ==str:
        if len(a.split(' '))==2:
            z=a.split(' ')[1]
        else:
            z=a.split(' ')[0]
    else :
        z=a
    return z


try:
    bal=[c for c in dt.columns if "BALANCE" in str(c).upper() ][0]
    dt[bal]=dt[bal].apply( lambda x: abc(x) )
except: print("\nBalance columns missing")

In [138]:
dt.head()

,Sl. No.,Date,Description,Chq / Ref number,Amount,Dr / Cr,Balance,Dr / Cr
0,,,IMPS from VARUN GHEI Ref,IMPS-912416820045,"10,000.00",CR,"69,300.17",CR
1,1,04/05/2019,,,,,,
2,,,912416332500,,,,,
3,,,MB:IMPS to 017101516788,IMPS-912416823590,"10,000.00",DR,"59,300.17",CR
4,2,04/05/2019,,,,,,


In [139]:
# dt['flag']=dt.iloc[:,-2].shift(1)
# dt=dt[~dt.index.isin(dt[dt.iloc[:,-1]==dt.iloc[:,-3]].index)]

In [140]:
# dt

In [141]:
try:
    amt=[c for c in dt.columns if "WITHDRAW" in str(c).upper() ][0]
except:
    try:
        amt=[c for c in dt.columns if "DEPOSIT" in str(c).upper() ][0]
    except:
        try:
            amt=[c for c in dt.columns if "AMOUNT" in str(c).upper() ][0]
        except: pass

In [142]:
dt.head()

,Sl. No.,Date,Description,Chq / Ref number,Amount,Dr / Cr,Balance,Dr / Cr
0,,,IMPS from VARUN GHEI Ref,IMPS-912416820045,"10,000.00",CR,"69,300.17",CR
1,1,04/05/2019,,,,,,
2,,,912416332500,,,,,
3,,,MB:IMPS to 017101516788,IMPS-912416823590,"10,000.00",DR,"59,300.17",CR
4,2,04/05/2019,,,,,,


In [143]:
dt[[bal,amt]]=dt[[bal,amt]].replace(r'^\s*$', np.nan, regex=True)

In [144]:
dt.head()

,Sl. No.,Date,Description,Chq / Ref number,Amount,Dr / Cr,Balance,Dr / Cr
0,,,IMPS from VARUN GHEI Ref,IMPS-912416820045,"10,000.00",CR,"69,300.17",CR
1,1,04/05/2019,,,NaN,,NaN,
2,,,912416332500,,NaN,,NaN,
3,,,MB:IMPS to 017101516788,IMPS-912416823590,"10,000.00",DR,"59,300.17",CR
4,2,04/05/2019,,,NaN,,NaN,


In [145]:
try:
    dt[amt]=dt[amt].apply( lambda x: x.split(' ')[0] if type(x) == str else x )                       
    dt['Wdl1']=dt[amt].astype(str).apply(lambda x: str(x).replace(",","").replace("(Cr)","").replace("(Dr)","")).astype(float)             
except Exception as e: 
    print("Error",e)

In [146]:
dt.head()

,Sl. No.,Date,Description,Chq / Ref number,Amount,Dr / Cr,Balance,Dr / Cr,Wdl1
0,,,IMPS from VARUN GHEI Ref,IMPS-912416820045,"10,000.00",CR,"69,300.17",CR,10000.0
1,1,04/05/2019,,,NaN,,NaN,,NaN
2,,,912416332500,,NaN,,NaN,,NaN
3,,,MB:IMPS to 017101516788,IMPS-912416823590,"10,000.00",DR,"59,300.17",CR,10000.0
4,2,04/05/2019,,,NaN,,NaN,,NaN


In [147]:
bal

'Balance'

In [148]:
dt.to_csv("parsed2.csv", index=0)

In [149]:
try:
    dt[bal]=dt[bal].apply(lambda x: str(x).replace(",","").replace("(Cr)","").replace("(Dr)","")).astype(float)
except Exception as e:
    pass
try:
    dt['flag']=dt['Wdl1'].astype(str) +dt[bal].astype(str)
except:pass
    
try:
    dt['flag2']=np.arange(len(dt))
except:pass

In [150]:
dt.head(20)

,Sl. No.,Date,Description,Chq / Ref number,Amount,Dr / Cr,Balance,Dr / Cr,Wdl1,flag,flag2
0,,,IMPS from VARUN GHEI Ref,IMPS-912416820045,"10,000.00",CR,"69,300.17",CR,10000.00,"10000.069,300.17",0
1,1,04/05/2019,,,NaN,,NaN,,NaN,nannan,1
2,,,912416332500,,NaN,,NaN,,NaN,nannan,2
3,,,MB:IMPS to 017101516788,IMPS-912416823590,"10,000.00",DR,"59,300.17",CR,10000.00,"10000.059,300.17",3
4,2,04/05/2019,,,NaN,,NaN,,NaN,nannan,4
5,,,Ref 912416823635,,NaN,,NaN,,NaN,nannan,5
6,,,CASH DEPOSIT AT 4611 BY,,"34,000.00",CR,"93,300.17",CR,34000.00,"34000.093,300.17",6
7,3,07/05/2019,,,NaN,,NaN,,NaN,nannan,7
8,,,AZAR,,NaN,,NaN,,NaN,nannan,8
9,,,MB:IMPS to 017101516788,IMPS-912711056281,"45,000.00",DR,"48,300.17",CR,45000.00,"45000.048,300.17",9


In [151]:
dt.loc[ dt[['flag']].duplicated(keep=False) , 'flag' ] =dt['flag'] + dt['flag2'].astype(str)  
dt['flag']=dt['flag'].apply( lambda row : np.nan if 'nannan' in row else row ).fillna(method='ffill')        
dt['Description']=dt.groupby('flag')['Description'].transform( lambda x :''.join(x)) 
dt['Chq / Ref number']=dt.groupby('flag')['Chq / Ref number'].transform( lambda x :''.join(x))
dt['Date']=dt.groupby('flag')['Date'].transform( lambda x :''.join(x))
dt = dt.drop_duplicates(['flag'], keep='first').iloc[0:,:-3].reset_index(drop=True)

In [152]:
dt.head(10)

,Sl. No.,Date,Description,Chq / Ref number,Amount,Dr / Cr,Balance,Dr / Cr
0,,04/05/2019,IMPS from VARUN GHEI Ref912416332500,IMPS-912416820045,"10,000.00",CR,"69,300.17",CR
1,,04/05/2019,MB:IMPS to 017101516788Ref 912416823635,IMPS-912416823590,"10,000.00",DR,"59,300.17",CR
2,,07/05/2019,CASH DEPOSIT AT 4611 BYAZAR,,"34,000.00",CR,"93,300.17",CR
3,,07/05/2019,MB:IMPS to 017101516788Ref 912711056460,IMPS-912711056281,"45,000.00",DR,"48,300.17",CR
4,,07/05/2019,MB:SENT TO A TRADING7212860439,MB-999617566531,"46,300.00",DR,"2,000.17",CR
5,,08/05/2019,PCD/1163/CITRUSPAYZOMATO/MUMBAI080519/19:55,912814114668,731.62,DR,"1,268.55",CR
6,,09/05/2019,PCD/1163/FUNKY LIL/NEWDELHI090519/17:28,912911956081,700.00,DR,568.55,CR
7,,09/05/2019,MB:IMPS to 0248100004876Ref 912920482876,IMPS-912920482878,300.00,DR,268.55,CR
8,,13/05/2019,Chrg: Daily Bal Alertscharges for Apr 19,TBMS,11.80,DR,256.75,CR
9,,15/05/2019,MB:RECEIVED FROMA TRADING,MB-999614168325,"50,000.00",CR,"50,256.75",CR


In [153]:
dt.to_csv("check.csv", index=0)

In [154]:
dt = dt.iloc[:,:-1]
dt.head()

,Sl. No.,Date,Description,Chq / Ref number,Amount,Dr / Cr,Balance
0,,04/05/2019,IMPS from VARUN GHEI Ref912416332500,IMPS-912416820045,"10,000.00",CR,"69,300.17"
1,,04/05/2019,MB:IMPS to 017101516788Ref 912416823635,IMPS-912416823590,"10,000.00",DR,"59,300.17"
2,,07/05/2019,CASH DEPOSIT AT 4611 BYAZAR,,"34,000.00",CR,"93,300.17"
3,,07/05/2019,MB:IMPS to 017101516788Ref 912711056460,IMPS-912711056281,"45,000.00",DR,"48,300.17"
4,,07/05/2019,MB:SENT TO A TRADING7212860439,MB-999617566531,"46,300.00",DR,"2,000.17"


In [155]:
try:
    drcr=[c for c in dt.columns if "DR" in str(c).upper() ][0]
    amt = [c for c in dt.columns if "AMOUNT" in str(c).upper()][0]
    print(drcr,amt)
    dt["Credits"] = ""
    dt["Debits"] = ""
except:pass

try:
    for i in dt.index:
        if dt[drcr][i] == "DR":
            dt["Debits"][i] = dt[amt][i]
        elif dt[drcr][i] == "CR":
            dt["Credits"][i] = dt[amt][i]
        else:
            print(i)
            print("Dr & CR Absent")
except:pass

Dr / Cr Amount


In [156]:
# dt = dt.replace(r'^\s*$', np.nan, regex=True)

# if dt[drcr][0] == "CR":
#     dt["Credits"] = np.nan
#     dt["Credits"][0] = float(str(dt[amt][0]).replace(",","").replace("(Cr)","").replace("(Dr)",""))
#     dt["Debits"] = np.nan
# elif dt[drcr][0] == "DR":
#     dt["Credits"] = np.nan    
#     dt["Debits"] = np.nan
#     dt["Debits"][0] = float(str(dt[amt][0]).replace(",","").replace("(Cr)","").replace("(Dr)","")) *-1

# for i,j in enumerate(dt[bal]):
#     if i < len(dt[bal])-1 :
#         if dt[bal][i] < dt[bal][i+1]:
#             dt["Credits"][i+1] = dt[bal][i+1]-dt[bal][i]
#         elif dt[bal][i] > dt[bal][i+1]:
#             dt["Debits"][i+1] = dt[bal][i+1]-dt[bal][i]
#     else:
#         pass

In [157]:
dt.head()


,Sl. No.,Date,Description,Chq / Ref number,Amount,Dr / Cr,Balance,Credits,Debits
0,,04/05/2019,IMPS from VARUN GHEI Ref912416332500,IMPS-912416820045,"10,000.00",CR,"69,300.17","10,000.00",
1,,04/05/2019,MB:IMPS to 017101516788Ref 912416823635,IMPS-912416823590,"10,000.00",DR,"59,300.17",,"10,000.00"
2,,07/05/2019,CASH DEPOSIT AT 4611 BYAZAR,,"34,000.00",CR,"93,300.17","34,000.00",
3,,07/05/2019,MB:IMPS to 017101516788Ref 912711056460,IMPS-912711056281,"45,000.00",DR,"48,300.17",,"45,000.00"
4,,07/05/2019,MB:SENT TO A TRADING7212860439,MB-999617566531,"46,300.00",DR,"2,000.17",,"46,300.00"


In [158]:
df = dt

In [159]:
try:
    dat=[c for c in df.columns if "TRANSACTION DATE" in str(c).upper() ][0]
except:
    try:
        dat=[c for c in df.columns if "TXN DATE" in str(c).upper() ][0]
    except:
        try:
            dat=[c for c in df.columns if "DATE" in str(c).upper() ][0]
        except:pass

try:
    chq=[c for c in df.columns if "CHQ" in str(c).upper() ][0]
except:
    try:
        chq=[c for c in df.columns if "CHEQUE" in str(c).upper() ][0]
    except:pass

try:
    narr=[c for c in df.columns if "REMARKS" in str(c).upper() ][0]
except:
    try:
        narr=[c for c in df.columns if "PARTICULARS" in str(c).upper() ][0]
    except:
        try:
            narr=[c for c in df.columns if "DESCRIPTION" in str(c).upper() ][0]
        except:
            try:
                narr=[c for c in df.columns if "DETAILS" in str(c).upper() ][0]
            except:pass

try:
    bal=[c for c in df.columns if "BALANCE" in str(c).upper() ][0]
    df["bal"]=df[bal].apply( lambda x: abc(x) )
except: print("\nBalance columns missing")
    
print(bal)

try:
    wdl=[c for c in df.columns if "WITHDRAW" in str(c).upper() ][0]
except:
    try:
        wdl=[c for c in df.columns if "DEBIT" in str(c).upper() ][0]
    except: pass 

try:
    dep=[c for c in df.columns if "DEPOSIT" in str(c).upper() ][0]
except:
    try:
        dep=[c for c in df.columns if "CREDIT" in str(c).upper() ][0]
    except: pass

print(wdl,dep)
    
df[[wdl, dep]] = df[[wdl, dep]].replace({"NA":np.nan, "-":np.nan,"0":np.nan,"":np.nan})

try:
    auto=[c for c in df.columns if "AUTOSWEEP" in str(c).upper() ][0]
    for j,i in enumerate(df[auto]):
        if isnan(i) == False:
            df[wdl][j] = df[auto][j]
        else:
            pass
except: pass

try:
    rev=[c for c in df.columns if "REVERSE" in str(c).upper() ][0]
    for j,i in enumerate(df[rev]):
        if isnan(i) == False:
            df[dep][j] = df[rev][j]
        else:
            pass
except: pass

try:
    df[dep]=df[dep].apply( lambda x: x.split(' ')[0] if type(x) == str else x )
except:pass
try:
    df[wdl]=df[wdl].apply( lambda x: x.split(' ')[0] if type(x) == str else x )
except:pass
# df.to_csv("check.csv", index=0)
try:
    df[wdl]=df[wdl].astype(str).apply(lambda x: str(x).replace("\r","").replace(",","").replace("Cr","").replace("Dr","")).astype(float) *-1
except:pass
try:
    df[dep]=df[dep].astype(str).apply(lambda x: str(x).replace("\r","").replace(",","").replace("Cr","").replace("Dr","")).astype(float)
except:pass
try:
    df[bal]=df[bal].astype(str).apply(lambda x:str(x).replace("\r","").replace(",","").replace("Cr","").replace("Dr","")).astype(float)
except:pass
print(df.head())

try:
    for i in df.index:
        if df["bal"][i] == "Dr":
            df[bal][i] = df[bal][i]*-1
        else:
            pass
except:pass

try:
    df.drop(['bal'], axis=1, inplace=True)
except:pass

try:
    df.drop([auto], axis=1, inplace=True)
except:pass

try:
    df.drop([rev], axis=1, inplace=True)
except:pass

df = df[[dat,chq,narr,wdl,dep,bal]]
df.columns = ["Xns Date","Cheque No","Narration","Debits","Credits","Balance"]

df.to_excel("check.xlsx", index=False)

Balance
Debits Credits
0 Sl. No.        Date                              Description  \
0          04/05/2019     IMPS from VARUN GHEI Ref912416332500   
1          04/05/2019  MB:IMPS to 017101516788Ref 912416823635   
2          07/05/2019              CASH DEPOSIT AT 4611 BYAZAR   
3          07/05/2019  MB:IMPS to 017101516788Ref 912711056460   
4          07/05/2019           MB:SENT TO A TRADING7212860439   

0   Chq / Ref number     Amount Dr / Cr    Balance  Credits   Debits  \
0  IMPS-912416820045  10,000.00      CR  69,300.17  10000.0      NaN   
1  IMPS-912416823590  10,000.00      DR  59,300.17      NaN -10000.0   
2                     34,000.00      CR  93,300.17  34000.0      NaN   
3  IMPS-912711056281  45,000.00      DR  48,300.17      NaN -45000.0   
4    MB-999617566531  46,300.00      DR   2,000.17      NaN -46300.0   

0        bal  
0  69,300.17  
1  59,300.17  
2  93,300.17  
3  48,300.17  
4   2,000.17  


In [ ]:
if len(tables) != 0:
    dt=pd.DataFrame(); tmp=pd.DataFrame()
    for i in range(len(tables)):
        tmp=tables[i].df
        if (tmp.shape[1]<5):
            pass
            #print("\nCustomer information structure found.",i); pass
        elif (tmp.shape[1]==5):
            try:
                idx=[ c for c in tmp[tmp.apply(lambda row: row.astype(str).str.lower().str.contains('deposit').any(), axis=1) ==True].index ][0]
                tmp=tmp.iloc[idx+1:,:] ; tmp.reset_index(drop=True,inplace=True)
            except:
                print("\nAxis-Column headers missing"); pass
            tmp.columns = ["Date","Narration","Cheque No","Amount","Balance"]
            #tmp=pd.DataFrame(tmp).replace(r'^\s*$', np.nan, regex=True)
            dt=pd.concat([dt,tmp]).reset_index(drop=True)
        else:
            pass

In [ ]:
dt.to_csv("check.csv", index=0)

In [ ]:
dt.drop_duplicates().reset_index(drop=True)

In [ ]:
def abc(a):
    if type(a) ==str:
        if len(a.split(' '))==2:
            z=a.split(' ')[1]
        else:
            z=a.split(' ')[0]
    else :
        z=a
    return z       

dt=dt[~dt.index.isin(dt[dt.apply(lambda row: row.astype(str).str.lower().str.contains('page:|account status|total|reason for return|inward clg|opening balance|statement of a/c|statement summary|generated on|generated by|computer generated statement|not').any(), axis=1) ==True].index)]
dt.drop(dt.nunique(dropna=False)[(dt.nunique(dropna=False) == 1)].index, axis=1,inplace=True)             
dt.iloc[:,-1]=dt.iloc[:,-1].fillna(dt.iloc[:,-2])

try:
    bal=[c for c in dt.columns if "BALANCE" in str(c).upper() ][0]
    dt[bal]=dt[bal].apply( lambda x: abc(x) )
except: print("\nBalance columns missing")           

dt['flag']=dt.iloc[:,-1].shift(1)
dt=dt[~dt.index.isin(dt[dt.iloc[:,-1]==dt.iloc[:,-2]].index)]

try:
    wdl=[c for c in dt.columns if "WITHDRAW" in str(c).upper() ][0]
except:
    try:
        wdl=[c for c in dt.columns if "AMOUNT" in str(c).upper() ][0]
    except: pass   

try:
    dep=[c for c in dt.columns if "DEPOSIT" in str(c).upper() ][0]
except: pass            

In [ ]:
wdl

In [ ]:
df = dt

In [ ]:
df[[bal,wdl]]=df[[bal,wdl]].replace(r'^\s*$', np.nan, regex=True)

In [ ]:
df.tail()

In [ ]:
try:
    df[wdl]=df[wdl].apply( lambda x: x.split(' ')[0] if type(x) == str else x )                       
    df['Wdl1']=df[wdl].astype(str).apply(lambda x: str(x).replace(",","").replace("(Cr)","").replace("(Dr)","")).astype(float) *-1               
except Exception as e: 
    print("except",e)
    try:
        Wdl1=[]
        for i, item in enumerate(df[wdl]):
            try:
                tmpp=float(str(item).replace(",","").replace("(Cr)","").replace("(Dr)","")) *-1
            except ValueError:
                df.iloc[i,df.columns.get_loc(wdl)-1]=df.iloc[i,df.columns.get_loc(wdl)]
                df.iloc[i,df.columns.get_loc(wdl)]=df.iloc[i,df.columns.get_loc(dep)]
                df.iloc[i,df.columns.get_loc(dep)]=df.iloc[i,df.columns.get_loc(bal)]
                df.iloc[i,df.columns.get_loc(bal)]=df.iloc[i,df.columns.get_loc(bal)+1]
            Wdl1.append(tmpp)  
        df[bal]=df[bal].fillna(df.iloc[:,df.columns.get_loc(bal)-1])
        df.drop(df.columns[df.columns.get_loc(bal)+1],axis=1,inplace=True)
        df['Wdl1']=Wdl1   
        df['Wdl1']=df['Wdl1'].fillna(df[dep].astype(str).apply(lambda x: str(x).replace(",","").replace("(Cr)","").replace("(Dr)","")).astype(float) )                
    except:
        print("except",e)
        pass 

In [ ]:
df.tail()

In [ ]:
df=df.T.drop_duplicates().T           
df[bal]=df[bal].apply(lambda x: str(x).replace(",","").replace("(Cr)","").replace("(Dr)","")).astype(float)                                 
df['flag']=df.iloc[:,0].astype(str)+df['Wdl1'].astype(str) +df[bal].astype(str)
df['flag2']=np.arange(len(df))

df.loc[ df[['flag']].duplicated(keep=False) , 'flag' ] =df['flag'] + df['flag2'].astype(str)  
df['flag']=df['flag'].apply( lambda row : np.nan if 'nannan' in row else row ).fillna(method='ffill')        

df.head()

In [ ]:
df.head(20)

In [ ]:
df['Narration']=df.groupby('flag')['Narration'].transform( lambda x :''.join(x)) 
df.head()

In [ ]:
df.head(20)

In [ ]:
df['Cheque No']=df.groupby('flag')['Cheque No'].transform( lambda x :''.join(x))
df.head()

In [ ]:
df=df.drop_duplicates(['flag'],keep='first').iloc[0:,:-3].reset_index(drop=True)

In [ ]:
df.head()

In [ ]:
df.to_csv("check.csv", index=0)

In [ ]:
bal

In [ ]:
df["Credits"] = np.nan
df["Debits"] = np.nan
for i,j in enumerate(df[bal]):
    if i < len(df[bal])-1 :
        if df[bal][i] < df[bal][i+1]:
            df["Credits"][i+1] = df[bal][i+1]-df[bal][i]
        elif df[bal][i] > df[bal][i+1]:
            df["Debits"][i+1] = df[bal][i+1]-df[bal][i]
    else:
        pass

In [ ]:
df.to_csv("check02.csv", index=0)